# To-Do: Analyse der Ladeinfrastruktur in Deutschland

This notebook analyzes and visualizes the current charging infrastructure for electric vehicles (EVs) in Germany. The remainder of this notebook is in German, as the target dashboard will also be in German.

---

### 1. Statistische Kennzahlen (KPIs)

- [ ] **1.1 Anzahl der Ladestationen:** Ermittle die Gesamtzahl aller einzigartigen Ladestationen.
- [ ] **1.2 Anzahl der Ladepunkte:** Ermittle die Gesamtzahl aller Ladepunkte.
- [ ] **1.3.1 Installierte Ladeleistung (nach Ladepunkt):** Berechne die Summe der `LadeleistungInKW`.
- [ ] **1.3.2 Installierte Ladeleistung (nach Ladesäule):** Berechne die Summe der `InstallierteLadeleistungNLL` pro einzigartiger Station.
- [ ] **1.4 Anzahl HPC-Ladepunkte:** Zähle die Anzahl der Hochleistungsladepunkte (z.B. >= 150 kW).
- [ ] **1.5 Wachstumsrate Ladepunkte ('24 vs. '25):** Berechne die prozentuale Veränderung.
- [ ] **1.6 Wachstumsrate Ladestationen ('24 vs. '25):** Berechne die prozentuale Veränderung.

---

### 2. Zeitreihen-Entwicklung

- [ ] **2.1 Kumulative Entwicklung Ladestationen:** Erstelle ein Liniendiagramm, das den kumulativen Zuwachs der Stationen über die Zeit zeigt.
- [ ] **2.2 Kumulative Entwicklung Ladepunkte:** Erstelle ein Liniendiagramm, das den kumulativen Zuwachs der Punkte über die Zeit zeigt.
- [ ] **2.3 Jährlicher Zubau (nicht-kumulativ):** Erstelle ein Balkendiagramm, das die Anzahl neuer Ladepunkte pro Jahr anzeigt.

---

### 3. Geografische Darstellung

- [ ] **3.1 Deutschlandkarte:** Erstelle eine Choroplethenkarte auf Kreisebene, welche die Dichte der Ladepunkte anzeigt.
- [ ] **3.2 Regionale Verteilung:** Erstelle ein Balkendiagramm mit der Anzahl der Ladepunkte pro Bundesland.

---

### 4. Detaillierte Analysen

- [ ] **4.1 Anteil an Schnellladern:** Visualisiere die Verteilung der Ladepunkttypen (Normal-, Schnell-, HPC-Lader).
- [ ] **4.2 Top-Betreiber:** Erstelle ein Balkendiagramm der 10 größten Betreiber nach Anzahl der Ladepunkte.
- [ ] **4.3 Lade-Anwendungsfälle:** Analysiere die Verteilung der Ladepunkte nach ihrem Standort (z.B. Straße, Einzelhandel).
- [ ] **4.4 Verteilung der Ladeleistungen:** Erstelle ein Histogramm, um die Verteilung der `LadeleistungInKW` zu zeigen.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from streamlit_folium import st_folium
import io
import plotly.express as px

2025-09-11 16:28:45.329 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [3]:

# Load the combined data which is on the Ladepunkt level
df = pd.read_csv("../02_data/03_computed_data/combined_ladestation_ladepunkt.csv")

df.shape
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171805 entries, 0 to 171804
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ladestation_id                 171805 non-null  int64  
 1   Betreiber                      171805 non-null  object 
 2   BetreiberBereinigt             171803 non-null  object 
 3   BetreiberKategorie             171805 non-null  object 
 4   Strasse                        171805 non-null  object 
 5   Hausnummer                     171805 non-null  object 
 6   Adresszusatz                   29126 non-null   object 
 7   PLZ                            171805 non-null  int64  
 8   Ort                            171805 non-null  object 
 9   Bundesland                     171805 non-null  object 
 10  KreisKreisfreieStadt           171805 non-null  object 
 11  Gemeinde                       171805 non-null  object 
 12  ARS                           

,ladestation_id,Betreiber,BetreiberBereinigt,BetreiberKategorie,Strasse,Hausnummer,Adresszusatz,PLZ,Ort,Bundesland,...,CSS_Stecker_Combo_2_Stecker,Tesla_Supercharger_V2_Stecker,Tesla_Supercharger_V3_Stecker,CHAdeMO_Stecker,LPArtLSV,LPArtHPC,LPArtFR,LPArtCharIN,Datenstand_y,ladepunkt_id
0,1,Albwerk Elektro- und Kommunikationstechnik GmbH,Albwerk Elektro- und Kommunikationstechnik,Sonstiges,Ennabeurer Weg,0,NaN,72535,Heroldstatt,Baden-Württemberg,...,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26,1
1,1,Albwerk Elektro- und Kommunikationstechnik GmbH,Albwerk Elektro- und Kommunikationstechnik,Sonstiges,Ennabeurer Weg,0,NaN,72535,Heroldstatt,Baden-Württemberg,...,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26,2
2,2,smopi® - Multi Chargepoint Solution GmbH,smopi - Multi Chargepoint Solution,Sonstiges,Albstraße,15,NaN,72535,Heroldstatt,Baden-Württemberg,...,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26,3
3,2,smopi® - Multi Chargepoint Solution GmbH,smopi - Multi Chargepoint Solution,Sonstiges,Albstraße,15,NaN,72535,Heroldstatt,Baden-Württemberg,...,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26,4
4,2,smopi® - Multi Chargepoint Solution GmbH,smopi - Multi Chargepoint Solution,Sonstiges,Albstraße,15,NaN,72535,Heroldstatt,Baden-Württemberg,...,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26,5


In [4]:
# --- Datenvorbereitung ---
df['Inbetriebnahmedatum'] = pd.to_datetime(df['Inbetriebnahmedatum'], errors='coerce')
df.dropna(subset=['Inbetriebnahmedatum'], inplace=True)

# --- Berechnungen (KPIs) ---
print("### 1. Statistische Kennzahlen (KPIs) 📈\n")

# 1.1 Anzahl der Ladestationen
num_ladestationen = df['ladestation_id'].nunique()
print(f"# 1.1 Anzahl der Ladestationen: {num_ladestationen}")

# 1.2 Anzahl der Ladepunkte
num_ladepunkte = len(df)
print(f"# 1.2 Anzahl der Ladepunkte: {num_ladepunkte}")

# 1.3.1 Installierte Ladeleistung nach Ladepunkt
leistung_ladepunkt_kw = df['LadeleistungInKW'].sum()
leistung_ladepunkt_gw = leistung_ladepunkt_kw / 1_000_000
print(f"# 1.3.1 Installierte Ladeleistung nach Ladepunkt: {leistung_ladepunkt_gw:.4f} GW")

# 1.3.2 Installierte Ladeleistung nach Ladestation (Ladesäule)
leistung_pro_station = df.groupby('ladestation_id')['InstallierteLadeleistungNLL'].first().sum()
leistung_station_gw = leistung_pro_station / 1_000_000
print(f"# 1.3.2 Installierte Ladeleistung nach Ladesäule: {leistung_station_gw:.4f} GW")

# 1.4 Anzahl HPC-Ladepunkte
hpc_ladepunkte = df[df['LadeleistungInKW'] >= 150]
num_hpc_ladepunkte = len(hpc_ladepunkte)
print(f"# 1.4 Anzahl HPC-Ladepunkte (>= 150 kW): {num_hpc_ladepunkte}")

# 1.5 & 1.6 Wachstumsraten
df_2024_or_earlier = df[df['Inbetriebnahmedatum'].dt.year <= 2024]
df_2025_or_earlier = df[df['Inbetriebnahmedatum'].dt.year <= 2025]

ladepunkte_ende_2024 = len(df_2024_or_earlier)
ladestationen_ende_2024 = df_2024_or_earlier['ladestation_id'].nunique()

ladepunkte_ende_2025 = len(df_2025_or_earlier)
ladestationen_ende_2025 = df_2025_or_earlier['ladestation_id'].nunique()

if ladepunkte_ende_2024 > 0:
    wachstumsrate_ladepunkte = ((ladepunkte_ende_2025 - ladepunkte_ende_2024) / ladepunkte_ende_2024) * 100
    print(f"# 1.5 Wachstumsrate Ladepunkte ('24 auf '25): {wachstumsrate_ladepunkte:.2f}%")

if ladestationen_ende_2024 > 0:
    wachstumsrate_ladestationen = ((ladestationen_ende_2025 - ladestationen_ende_2024) / ladestationen_ende_2024) * 100
    print(f"# 1.6 Wachstumsrate Ladestationen ('24 auf '25): {wachstumsrate_ladestationen:.2f}%")


### 1. Statistische Kennzahlen (KPIs) 📈

# 1.1 Anzahl der Ladestationen: 94069
# 1.2 Anzahl der Ladepunkte: 171805
# 1.3.1 Installierte Ladeleistung nach Ladepunkt: 10.2231 GW
# 1.3.2 Installierte Ladeleistung nach Ladesäule: 6.8855 GW
# 1.4 Anzahl HPC-Ladepunkte (>= 150 kW): 29109
# 1.5 Wachstumsrate Ladepunkte ('24 auf '25): 8.53%
# 1.6 Wachstumsrate Ladestationen ('24 auf '25): 8.71%


In [ ]:
# --- Zeitreihen-Entwicklung mit Plotly ---
print("\n### 2. Interaktive Zeitreihen-Entwicklung mit Plotly 📊\n")
print("Die interaktiven Plots werden generiert und als HTML-Dateien gespeichert.")

df_sorted = df.sort_values('Inbetriebnahmedatum')

# --- Plotting mit Plotly und angepasster X-Achse ---

# Plot 2.1: Entwicklung Ladestationen
fig_stationen = px.line(
    cumulative_ladestationen,
    x='Inbetriebnahmedatum',
    y='Anzahl',
    markers=True,
    labels={'Inbetriebnahmedatum': 'Datum der Inbetriebnahme', 'Anzahl': 'Anzahl der Ladestationen'}
)
fig_stationen.update_layout(
    title_text='<b>Kumulative Entwicklung der Ladestationen in Deutschland</b>',
    title_x=0.5,
    template='plotly_white',
    # --- NEU: X-Achse begrenzen ---
    xaxis_range=['2010-01-01','2025-12-31']
)
fig_stationen.write_html("entwicklung_ladestationen_interaktiv_2010-2025.html")
print("- Interaktiver Plot (2010-2025) 'entwicklung_ladestationen_interaktiv_2010-2025.html' wurde gespeichert.")

# Plot 2.2: Entwicklung Ladepunkte
fig_punkte = px.line(
    cumulative_ladepunkte,
    x='Inbetriebnahmedatum',
    y='Anzahl',
    markers=True,
    labels={'Inbetriebnahmedatum': 'Datum der Inbetriebnahme', 'Anzahl': 'Anzahl der Ladepunkte'}
)
fig_punkte.update_layout(
    title_text='<b>Kumulative Entwicklung der Ladepunkte in Deutschland</b>',
    title_x=0.5,
    template='plotly_white',
    # --- NEU: X-Achse begrenzen ---
    xaxis_range=['2010-01-01','2025-12-31']
)

fig_punkte.write_html("entwicklung_ladepunkte_interaktiv_2010-2025.html")
print("- Interaktiver Plot (2010-2025) 'entwicklung_ladepunkte_interaktiv_2010-2025.html' wurde gespeichert.")



# Erstelle das Balkendiagramm
fig_zubau = px.bar(
    zubau_gefiltert,
    x='Jahr',
    y='Anzahl',
    text_auto=True, # Zeigt die Werte direkt auf den Balken an
    labels={'Jahr': 'Jahr der Inbetriebnahme', 'Anzahl': 'Anzahl neuer Ladepunkte'}
)

# Update des Layouts für Titel etc.
fig_zubau.update_layout(
    title_text='<b>Jährlicher Zubau von Ladepunkten in Deutschland (2010-2025)</b>',
    title_x=0.5,
    template='plotly_white',
    xaxis={'type': 'category'} # Stellt sicher, dass die Jahre als Kategorien behandelt werden
)

# Speichere den Plot als interaktive HTML-Datei
fig_zubau.write_html("jaehrlicher_zubau_interaktiv_2010-2025.html")
print("- Interaktiver Plot (2010-2025) 'jaehrlicher_zubau_interaktiv_2010-2025.html' wurde gespeichert.")


### 2. Interaktive Zeitreihen-Entwicklung mit Plotly 📊

Die interaktiven Plots werden generiert und als HTML-Dateien gespeichert.
- Interaktiver Plot (2010-2025) 'entwicklung_ladestationen_interaktiv_2010-2025.html' wurde gespeichert.
- Interaktiver Plot (2010-2025) 'entwicklung_ladepunkte_interaktiv_2010-2025.html' wurde gespeichert.


NameError: name 'zubau_gefiltert' is not defined

In [18]:
import pandas as pd
import plotly.express as px

# Annahme: Der DataFrame 'df' wurde bereits geladen und das 'Inbetriebnahmedatum' konvertiert.
# df['Inbetriebnahmedatum'] = pd.to_datetime(df['Inbetriebnahmedatum'], errors='coerce')
# df.dropna(subset=['Inbetriebnahmedatum'], inplace=True)

# --- Datenaufbereitung für Plot 2.3 ---

# Berechne die Anzahl neuer Ladepunkte pro Jahr
zubau_pro_jahr = df.groupby(df['Inbetriebnahmedatum'].dt.year)['ladepunkt_id'].count().reset_index()
zubau_pro_jahr.rename(columns={'Inbetriebnahmedatum': 'Jahr', 'ladepunkt_id': 'Anzahl'}, inplace=True)

# Filtere die Daten auf den gewünschten Zeitraum (2010-2025)
zubau_gefiltert = zubau_pro_jahr[(zubau_pro_jahr['Jahr'] >= 2010) & (zubau_pro_jahr['Jahr'] <= 2025)]


# --- Plotting für Plot 2.3 ---

# Erstelle das Balkendiagramm
fig_zubau = px.bar(
    zubau_gefiltert,
    x='Jahr',
    y='Anzahl',
    text_auto=True, # Zeigt die Werte direkt auf den Balken an
    labels={'Jahr': 'Jahr der Inbetriebnahme', 'Anzahl': 'Anzahl neuer Ladepunkte'}
)

# Update des Layouts für Titel etc.
fig_zubau.update_layout(
    title_text='<b>Jährlicher Zubau von Ladepunkten in Deutschland (2010-2025)</b>',
    title_x=0.5,
    template='plotly_white',
    xaxis={'type': 'category'} # Stellt sicher, dass die Jahre als Kategorien behandelt werden
)

# Speichere den Plot als interaktive HTML-Datei
fig_zubau.write_html("jaehrlicher_zubau_interaktiv_2010-2025.html")
print("- Interaktiver Plot (2010-2025) 'jaehrlicher_zubau_interaktiv_2010-2025.html' wurde gespeichert.")

- Interaktiver Plot (2010-2025) 'jaehrlicher_zubau_interaktiv_2010-2025.html' wurde gespeichert.


In [19]:


# --- Datenaufbereitung für den Plot ---

# 1. Daten auf die richtige Ebene bringen: Jede Station nur einmal zählen
df_stationen = df.drop_duplicates(subset='ladestation_id')

# 2. Anzahl neuer Stationen pro Jahr berechnen
zubau_pro_jahr_stationen = df_stationen.groupby(df_stationen['Inbetriebnahmedatum'].dt.year).size().reset_index()
zubau_pro_jahr_stationen.rename(columns={0: 'Anzahl', 'Inbetriebnahmedatum': 'Jahr'}, inplace=True)

# 3. Daten auf den gewünschten Zeitraum (2010-2025) filtern
zubau_stationen_gefiltert = zubau_pro_jahr_stationen[(zubau_pro_jahr_stationen['Jahr'] >= 2010) & (zubau_pro_jahr_stationen['Jahr'] <= 2025)]


# --- Plotting ---

# Erstelle das Balkendiagramm
fig_zubau_stationen = px.bar(
    zubau_stationen_gefiltert,
    x='Jahr',
    y='Anzahl',
    text_auto=True,
    labels={'Jahr': 'Jahr der Inbetriebnahme', 'Anzahl': 'Anzahl neuer Ladestationen'}
)

# Update des Layouts für Titel etc.
fig_zubau_stationen.update_layout(
    title_text='<b>Jährlicher Zubau von Ladestationen in Deutschland (2010-2025)</b>',
    title_x=0.5,
    template='plotly_white',
    xaxis={'type': 'category'}
)
fig_zubau_stationen.update_traces(marker_color='blue') # Farbe zur Unterscheidung ändern

# Speichere den Plot als interaktive HTML-Datei
fig_zubau_stationen.write_html("jaehrlicher_zubau_stationen_interaktiv_2010-2025.html")
print("- Interaktiver Plot (2010-2025) 'jaehrlicher_zubau_stationen_interaktiv_2010-2025.html' wurde gespeichert.")

- Interaktiver Plot (2010-2025) 'jaehrlicher_zubau_stationen_interaktiv_2010-2025.html' wurde gespeichert.


In [8]:

# Zähle die Anzahl der Ladepunkte (Zeilen) pro Bundesland
verteilung_bundesland = df['Bundesland'].value_counts().reset_index()
# Benenne die Spalten für eine saubere Beschriftung im Plot
verteilung_bundesland.columns = ['Bundesland', 'Anzahl Ladepunkte']


# --- Plotting ---

# Erstelle das Balkendiagramm
fig_bundesland = px.bar(
    verteilung_bundesland,
    x='Bundesland',
    y='Anzahl Ladepunkte',
    title='<b>3.2 Anzahl der Ladepunkte pro Bundesland</b>',
    labels={'Anzahl Ladepunkte': 'Anzahl der Ladepunkte', 'Bundesland': 'Bundesland'}
)

# Sortiere die X-Achse nach der Anzahl der Ladepunkte (absteigend)
fig_bundesland.update_xaxes(categoryorder='total descending')

# Speichere den Plot als interaktive HTML-Datei
fig_bundesland.write_html("3_2_regionale_verteilung.html")
print("- Interaktiver Plot '3_2_regionale_verteilung.html' wurde gespeichert.")


# --- Datenaufbereitung ---

# 1. Daten auf die Ebene der Stationen bringen (jede Station nur einmal)
df_stationen = df.drop_duplicates(subset='ladestation_id')

# 2. Zähle die Anzahl der einzigartigen Stationen pro Bundesland
verteilung_stationen = df_stationen['Bundesland'].value_counts().reset_index()
# Benenne die Spalten für eine saubere Beschriftung im Plot
verteilung_stationen.columns = ['Bundesland', 'Anzahl Ladestationen']


# --- Plotting ---

# Erstelle das Balkendiagramm
fig_stationen_bundesland = px.bar(
    verteilung_stationen,
    x='Bundesland',
    y='Anzahl Ladestationen',
    title='<b>3.3 Anzahl der Ladestationen pro Bundesland</b>',
    labels={'Anzahl Ladestationen': 'Anzahl der Ladestationen', 'Bundesland': 'Bundesland'}
)

# Sortiere die X-Achse nach der Anzahl der Ladestationen (absteigend)
fig_stationen_bundesland.update_xaxes(categoryorder='total descending')
fig_stationen_bundesland.update_traces(marker_color='purple') # Farbe zur Unterscheidung

# Speichere den Plot als interaktive HTML-Datei
fig_stationen_bundesland.write_html("3_3_stationen_pro_bundesland.html")
print("- Interaktiver Plot '3_3_stationen_pro_bundesland.html' wurde gespeichert.")

- Interaktiver Plot '3_2_regionale_verteilung.html' wurde gespeichert.
- Interaktiver Plot '3_3_stationen_pro_bundesland.html' wurde gespeichert.


In [ ]:

# --- 4.1 Anteil an Schnellladern ---

# Kategorien für die Ladeleistung erstellen
def get_leistungskategorie(leistung):
    if leistung >= 150:
        return 'HPC-Laden (>= 150 kW)'
    elif leistung > 22:
        return 'Schnellladen (> 22 kW)'
    else:
        return 'Normalladen (<= 22 kW)'

df['Leistungskategorie'] = df['LadeleistungInKW'].apply(get_leistungskategorie)
kategorie_counts = df['Leistungskategorie'].value_counts().reset_index()

# --- NEU: Eine Farbpalette definieren ---
color_map = {
    'Normalladen (<= 22 kW)': 'lightgrey',
    'Schnellladen (> 22 kW)': 'cornflowerblue',
    'HPC-Laden (>= 150 kW)': 'royalblue'
}

fig_kategorien = px.pie(
    kategorie_counts,
    names='Leistungskategorie',
    values='count',
    title='<b>4.1 Anteil der Ladepunkttypen in Deutschland</b>',
    # --- NEU: Die Farb-Karte anwenden ---
    color='Leistungskategorie',
    color_discrete_map=color_map
)
fig_kategorien.write_html("4_1_anteil_schnelllader.html")
print("- Plot 4.1 'anteil_schnelllader.html' wurde gespeichert.")


# --- 4.2 Top-Betreiber ---

top_10_betreiber = df['BetreiberBereinigt'].value_counts().nlargest(10).reset_index()

fig_betreiber = px.bar(
    top_10_betreiber,
    x='count',
    y='BetreiberBereinigt',
    orientation='h',
    title='<b>4.2 Top 10 Betreiber nach Anzahl der Ladepunkte</b>',
    labels={'count': 'Anzahl der Ladepunkte', 'BetreiberBereinigt': 'Betreiber'}
)
fig_betreiber.update_layout(yaxis={'categoryorder':'total ascending'}) # Sortiert die Balken
fig_betreiber.write_html("4_2_top_betreiber.html")
print("- Plot 4.2 'top_betreiber.html' wurde gespeichert.")


# --- 4.3 Lade-Anwendungsfälle ---

use_case_counts = df['LadeUseCase'].value_counts().reset_index()

fig_use_case = px.bar(
    use_case_counts,
    x='LadeUseCase',
    y='count',
    title='<b>4.3 Verteilung der Ladepunkte nach Anwendungsfall</b>',
    labels={'count': 'Anzahl der Ladepunkte', 'LadeUseCase': 'Anwendungsfall'}
)
fig_use_case.update_xaxes(categoryorder='total descending')
fig_use_case.write_html("4_3_lade_anwendungsfaelle.html")
print("- Plot 4.3 'lade_anwendungsfaelle.html' wurde gespeichert.")


# --- 4.4 Verteilung der Ladeleistungen ---

fig_histogramm = px.histogram(
    df,
    x='LadeleistungInKW',
    nbins=100, # Anzahl der "Balken" im Histogramm
    title='<b>4.4 Verteilung der Ladeleistungen (Histogramm)</b>',
    labels={'LadeleistungInKW': 'Ladeleistung in kW'}
)
# Begrenze die X-Achse für eine bessere Lesbarkeit (ignoriert extreme Ausreißer)
fig_histogramm.update_xaxes(range=[0, 400])
fig_histogramm.write_html("4_4_verteilung_ladeleistung.html")
print("- Plot 4.4 'verteilung_ladeleistung.html' wurde gespeichert.")

- Plot 4.1 'anteil_schnelllader.html' wurde gespeichert.
- Plot 4.2 'top_betreiber.html' wurde gespeichert.
- Plot 4.3 'lade_anwendungsfaelle.html' wurde gespeichert.
- Plot 4.4 'verteilung_ladeleistung.html' wurde gespeichert.
